In [3]:
from datetime import datetime
from pathlib import Path

from loguru import logger

from multiplex_pipeline.core_preparation.channel_scanner import (
                                build_transfer_map,
                                discover_channels,
)
from multiplex_pipeline.core_preparation.controller import (
                                CorePreparationController,
)
from multiplex_pipeline.core_preparation.file_io import GlobusFileStrategy
from multiplex_pipeline.utils.utils import load_analysis_settings

### Load analysis settings

In [7]:
# load analysis configuration
settings_path = r'R:\Wayne\BLCA\BLCA-2_Analysis\analysis_settings_BLCA2.yaml'

settings = load_analysis_settings(settings_path)
settings

{'image_dir': '/CellDive/BLCA-2/BLCA-2_Final',
 'analysis_dir': 'R:/Wayne/BLCA/BLCA-2_Analysis',
 'log_dir': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/logs'),
 'detection_image': 'BLCA-2_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'core_info_file_path': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/cores.csv'),
 'cores_dir_tif': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/temp'),
 'cores_dir_output': WindowsPath('R:/Wayne/BLCA/BLCA-2_Analysis/cores'),
 'include_channels': None,
 'exclude_channels': ['008_ECad'],
 'use_markers': None,
 'ignore_markers': ['Antibody1',
  'TNFa',
  'Snail1',
  'SKP2',
  'ProgRc',
  'Plk1',
  'PH3',
  'PDL1',
  'p65',
  'p130',
  'p-p130',
  'p-Cdc6',
  'LAG3',
  'IL-8',
  'HER2',
  'ERa',
  'EpCAM',
  'E2F1',
  'cycD3',
  'cycB2',
  'CDC25C',
  'CD86',
  'CD73',
  'CD69',
  'CD62L',
  'CD56',
  'CD4',
  'CD25',
  'CD19',
  'CD27',
  'CCR7',
  'cCASP3'],
 'segmentation': {'package': 'instaseg',
  'model': 'fluorescence_nuclei_and_cells',
  'kwargs': {'pixel_size': 0.3

### Define the logger

In [9]:
log_file = settings['log_dir'] / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

4

## Files through Globus

In [5]:
from multiplex_pipeline.utils.globus_utils import (
                                GlobusConfig,
                                create_globus_tc,
)

In [8]:
# get globus config
config_dir = Path(r'D:\\globus_config')
gc = GlobusConfig.from_config_files(config_dir)
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [ ]:
channel_map = discover_channels(Path(settings['image_dir']),
                                include_channels=settings['include_channels'],
                                exclude_channels=settings['exclude_channels'],
                                use_markers=settings.get('use_markers'),
                                ignore_markers=settings.get('ignore_markers'),
                                gc=gc)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (333695068.py, line 5)

In [13]:
selected_keys = ["DAPI", "CD3"]
short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
short_map

{'DAPI': '/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': '/CellDive/BLCA-1B/BLCA-1B_Final/BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [16]:
# build transfer map
transfer_cache_dir = Path(temp_dir) / "input_cache"
transfer_map = build_transfer_map(short_map, transfer_cache_dir)

# build a dict for transfered images
image_paths = {
    ch: str(Path(transfer_cache_dir) / Path(remote).name)
    for ch, (remote, _) in transfer_map.items()
}
image_paths

{'DAPI': 'D:\\multiplex_pipeline\\tests\\input_cache\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'D:\\multiplex_pipeline\\tests\\input_cache\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [36]:
strategy = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = transfer_cleanup_enabled)

controller = CorePreparationController(
    metadata_df = df.loc[15:15,:],
    image_paths = image_paths,
    temp_dir = temp_dir,
    output_dir = output_dir,
    file_strategy = strategy,
    margin = margin,
    mask_value = mask_value,
    max_pyramid_levels = max_pyramid_levels,
    core_cleanup_enabled = core_cleanup_enabled,
)

controller.run()

2025-06-17 16:32:35.669 | INFO     | multiplex_pipeline.core_preparation.file_io:_submit_all_transfers:65 - Submitted transfer for DAPI to /~/D/multiplex_pipeline/tests/input_cache/BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif (task_id=32598a4f-4bba-11f0-a701-0affcfc1d1e5)
2025-06-17 16:32:36.056 | INFO     | multiplex_pipeline.core_preparation.file_io:_submit_all_transfers:65 - Submitted transfer for CD3 to /~/D/multiplex_pipeline/tests/input_cache/BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif (task_id=32b53b46-4bba-11f0-9834-0affcfc1d1e5)
2025-06-17 16:32:36.097 | INFO     | multiplex_pipeline.core_preparation.controller:run:48 - Starting controller run loop...
2025-06-17 16:33:16.366 | INFO     | multiplex_pipeline.core_preparation.file_io:fetch_or_wait:100 - Transfer for CD3 complete: D:\multiplex_pipeline\tests\input_cache\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif
2025-06-17 16:33:17.608 | DEBUG    | multiplex_pipeline.core_preparation.controller:cut_channel:81 - Cu

INFO     The Zarr backing store has been changed from None the new file path:                                      
         R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015.zarr                                                         


2025-06-17 16:34:18.002 | DEBUG    | multiplex_pipeline.core_preparation.assembler:_cleanup_core_files:89 - Deleted intermediate TIFF: R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015\CD3.tiff
2025-06-17 16:34:18.068 | DEBUG    | multiplex_pipeline.core_preparation.assembler:_cleanup_core_files:89 - Deleted intermediate TIFF: R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015\DAPI.tiff
2025-06-17 16:34:18.068 | INFO     | multiplex_pipeline.core_preparation.controller:run:67 - All channels processed and cores assembled.
